In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import skew
import numpy as np
import seaborn as sns
import glob
import re

In [2]:
#!rm -r Results
!g++ Proyect.cpp -o ICDF.out
#!mkdir Results

In [3]:
#Corremos la simulacion
!./ICDF.out

60 0
60 10000
60 20000
60 30000
60 40000
70 0
70 10000
70 20000
70 30000
70 40000
80 0
80 10000
80 20000
80 30000
80 40000
90 0
90 10000
90 20000
90 30000
90 40000
100 0
100 10000
100 20000
100 30000
100 40000
110 0
110 10000
110 20000
110 30000
110 40000
120 0
120 10000
120 20000
120 30000
120 40000
130 0
130 10000
130 20000
130 30000
130 40000
140 0
140 10000
140 20000
140 30000
140 40000
150 0
150 10000
150 20000
150 30000
150 40000


# Analisis de tendencia

In [ ]:
#Lectura Alternativa del DF
DF = pd.read_csv('./Results/_South_180_Complete_trend_analysis.dat', sep = ' ')

In [ ]:
#Sacamos el N mas frecuente
N_most_frequent = DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']]
N = N_most_frequent["#Autos"].mode()[0]
iterations = N_most_frequent[N_most_frequent['#Autos'] == N].Iteration.unique()

In [ ]:
print(DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']].mean())
print(DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']].std())

In [ ]:
DF.groupby('Iteration').max().reset_index()[['Iteration','#Autos']]['#Autos'].value_counts()

In [ ]:
#Seleccionamos todos los procesos que contengan al N mas frecuente
DF_ = DF[DF.Iteration.isin(iterations)]

In [ ]:
#Revisamos las estadisticas de los resultados 
stats = pd.pivot_table(DF_, index = 'Iteration', values = 'Delta_t', aggfunc = [np.mean,np.std, np.median, skew])

In [ ]:
stats.sort_values(('std','Delta_t'))

In [ ]:
#Seleccionamos una simulacion que tenga resultados mas cercanos a los reales

In [ ]:
DF_ = DF_[DF_.Iteration.isin([25930])]

In [ ]:
#Hacemos el trend analysis de la region North
#Ajustamos Minimos cuadrados
x = np.array(DF_['#Autos']).reshape(-1, 1)
y = np.array(DF_['Delta_t']).reshape(-1, 1)
linreg = LinearRegression().fit(x,y)

In [ ]:
print('Intercept: {}'.format(linreg.intercept_))
print('Coef: {}'.format(linreg.coef_))

In [ ]:
plt.scatter(x,y)
ax = plt.gca()
ax.set_xlabel('Vehicle Count')
ax.set_ylabel('Time Intervals')
plt.plot(x,linreg.intercept_+linreg.coef_*x, color='r')

# Analisis de colas

In [4]:
colas = pd.read_csv('Cola_final.dat', sep = " ")

In [5]:
colas['sum'] = colas[['N','S','W','E']].sum(axis = 1)

In [16]:
colas.groupby('Tg').max()#.agg(pd.Series.mode)

,N,S,W,E,sum
Tg,,,,,
60,4920.0,6000.0,4560.0,203.0,15214.0
70,4095.0,4914.0,3958.5,93.5,12832.0
80,4212.0,4446.0,3432.0,65.0,11559.0
90,3575.0,4235.0,2667.5,34.0,10319.5
100,2925.0,3375.0,2115.0,19.0,8418.0
110,2700.0,3150.0,2137.5,11.0,7989.0
120,2376.0,2916.0,2052.0,8.0,7129.0
130,1904.0,2268.0,1554.0,5.5,5474.5
140,2156.0,2660.0,1764.0,6.0,6580.0


In [ ]:
colas.sum(axis = 1).hist()

In [ ]:
colas.sum(axis = 1).max()

In [ ]:
colas.sum(axis = 1).value_counts()

# NO BORRAR

#Creamos lista de DF
DF = pd.DataFrame()
for i in glob.glob('./Results/*.dat'):
    data = pd.read_csv(i, sep = " ")
    data['Iteration'] = int(re.findall(r'\d+',i)[0])
    DF =  pd.concat([DF,data],ignore_index=True)